In [1]:
import sys

from httpx import delete
from sqlalchemy import create_engine,text
import pandas as pd
import urllib
import os
import logging
# logging.basicConfig(
#     filename='D:/DA project/sales-analytics-project/notebooks/exploration/log_pip',
#     filemode='w',
#     level='INFO',
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#     datefmt='%d-%b-%y %H:%M:%S',
# )
logger=logging.getLogger("pipeline logger")
handler = logging.FileHandler(filename='pipeline logs.txt', mode='w')
logger.setLevel(logging.INFO)
logger.addHandler(handler)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)


In [6]:

server = ".\\SQLEXPRESS"
database = "sales-analytics"

params = urllib.parse.quote_plus(
    f"DRIVER=ODBC Driver 17 for SQL Server;"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print(result.scalar())


1


In [57]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
DATA_DIR = os.path.join(BASE_DIR, "data", "clean")
ITEMS_FILE = os.path.join(DATA_DIR, "order_items_clean.csv")
print(BASE_DIR)
print(DATA_DIR)
print(ITEMS_FILE)


D:\DA project\sales-analytics-project\notebooks
D:\DA project\sales-analytics-project\notebooks\data\clean
D:\DA project\sales-analytics-project\notebooks\data\clean\order_items_clean.csv


In [13]:
try:
    if os.path.exists("D:/DA project/sales-analytics-project/data/clean/"):
        for file in os.listdir("D:/DA project/sales-analytics-project/data/clean/"):
            if file.startswith("order_items_clean") and file.endswith(".csv"):
                #path=os.path.join("D:/DA project/sales-analytics-project/data/clean/",file)
                logger.info("loading cleaned order items data from directory")
                order_items = pd.read_csv(
                    parse_dates=["shipping_limit_date"]
                )
                logger.info(f"loaded cleaned order items data into order_items dataframe | rows={order_items.shape[0]}")
                print(order_items.head())
except Exception as e:
    logger.error(f"Failed to load order items data due to:{e}")
    print(e)



read_csv() missing 1 required positional argument: 'filepath_or_buffer'


In [71]:
delete(order_items)

NameError: name 'delete' is not defined

In [8]:

order_items = pd.read_csv(
    "D:/DA project/sales-analytics-project/data/clean/order_items_clean.csv",
    parse_dates=["shipping_limit_date"]
)

print("Rows in order_items CSV:", len(order_items))
order_items.head(2)

Rows in order_items CSV: 112650


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,is_zero_price,is_negative_price,is_high_freight
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,False,False,False
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93,False,False,False


In [9]:
fact_order_items_cols = [
    "order_id",
    "order_item_id",
    "product_id",
    "seller_id",
    "price",
    "freight_value",
    "shipping_limit_date",
    "is_zero_price",
    "is_negative_price",
    "is_high_freight"
]

fact_order_items_df = order_items[fact_order_items_cols]


In [10]:
fact_order_items_df.isnull().sum()


order_id               0
order_item_id          0
product_id             0
seller_id              0
price                  0
freight_value          0
shipping_limit_date    0
is_zero_price          0
is_negative_price      0
is_high_freight        0
dtype: int64

In [11]:
fact_order_items_df.to_sql(
    name="fact_order_items",
    con=engine,
    schema="dbo",
    if_exists="append",
    index=False,
    chunksize=1000
)
print("Loaded rows:", len(fact_order_items_df))
print("Distinct orders:", fact_order_items_df["order_id"].nunique())

Loaded rows: 112650
Distinct orders: 98666
